In [1]:
import numpy as np
import warnings 
import sys 
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:
    warnings.simplefilter('ignore')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data, meta = scipy.io.arff.loadarff('mtr-datasets/andro.arff')
data = pd.DataFrame(data)
data.head()

,Window0-Att0,Window0-Att1,Window0-Att2,Window0-Att3,Window0-Att4,Window0-Att5,Window1-Att0,Window1-Att1,Window1-Att2,Window1-Att3,...,Window4-Att2,Window4-Att3,Window4-Att4,Window4-Att5,Target,Target_2,Target_3,Target_4,Target_5,Target_6
0,11.84,5.08,49.0,32.2,16.5,1.40,12.01,5.09,48.0,32.0,...,45.0,29.7,38.5,3.27,15.17,5.14,46.0,30.4,46.8,3.74
1,12.01,5.09,48.0,32.0,14.8,1.25,12.89,5.10,48.0,31.9,...,46.0,30.1,42.6,3.55,14.85,5.11,47.0,31.2,46.8,3.78
2,12.89,5.10,48.0,31.9,8.6,0.71,13.95,5.11,47.0,30.8,...,48.0,31.3,46.0,3.79,14.72,5.10,48.0,31.7,42.7,3.40
3,13.95,5.11,47.0,30.8,20.2,1.64,13.75,5.13,45.0,29.7,...,47.0,31.1,46.7,3.79,15.28,5.11,48.0,32.0,42.5,3.38
4,13.75,5.13,45.0,29.7,38.5,3.27,13.34,5.13,46.0,30.1,...,45.0,29.7,45.1,3.75,16.15,5.13,47.0,30.8,42.1,3.27


In [3]:
X = data.iloc[:,:-6].values
y = data.iloc[:,-6:].values

scaler = StandardScaler()

X = scaler.fit_transform(X)
y = scaler.fit_transform(y)

print(X.shape,y.shape)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=100)

(49, 30) (49, 6)


In [4]:
print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)

(34, 30) (34, 6) (15, 30) (15, 6)


In [5]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,f1_score,r2_score
def evaluate(y_train,y_test):
    print("RMSE: ", mean_squared_error(y_train,y_test)**0.5)
    print("MAE: ", mean_absolute_error(y_train,y_test))
    print("R_Squared: ", r2_score(y_train,y_test))
    
    r2 = r2_score(y_train,y_test)
    
    n = 49
    p = 30

    adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    print('Adjusted R-squared - ',adj_r2)

### Bidirectional Model

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [7]:
ENS_COUNT = 4

In [8]:
def create_c1node(X_feed,y_now):
    model = Sequential()
    model.add(Dense(128,activation='relu',input_shape=(X_feed.shape[1],),kernel_initializer='he_normal'))
    model.add(Dense(64,activation='relu',kernel_initializer='he_normal'))
    model.add(Dense(y_now.shape[1],activation='linear',kernel_initializer='he_normal'))
    model.compile(loss='mse',optimizer='adam',metrics=['mse'])
    
    return model

### Forward Ensemble

In [9]:
ensembles_fwd = []
for j in range(ENS_COUNT//2):
    print('Training Ensemble {} - \n'.format(j))
    chain_forward = []
    
    X_feed = X_train.copy()
    y_t = y_train.copy()
    
    for i in range(y_t.shape[1]):
        print('Training chain node ',i)
        y_now = y_t[:,[i,]].copy()
        print('Shapes - \n X = {} \n Y = {}'.format(X_feed.shape,y_now.shape))
        
        node = create_c1node(X_feed,y_now)
        node.fit(X_feed,y_now,epochs=500,batch_size=50)
        print('Training of node {} complete\n'.format(i))
        
        chain_forward.append(node)
        X_feed = np.append(X_feed,y_now,axis=1)
        
    ensembles_fwd.append(chain_forward)

Training Ensemble 0 - 

Training chain node  0
Shapes - 
 X = (34, 30) 
 Y = (34, 1)





Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Epoch 1/500
34/34 [==============================] - 0s - loss: 1.5348 - mean_squared_error: 1.5348
Epoch 2/500
34/34 [==============================] - 0s - loss: 0.9321 - mean_squared_error: 0.9321
Epoch 3/500
34/34 [==============================] - 0s - loss: 0.5948 - mean_squared_error: 0.5948
Epoch 4/500
34/34 [==============================] - 0s - loss: 0.4421 - mean_squared_error: 0.4421
Epoch 5/500
34/34 [==============================] - 0s - loss: 0.4123 - mean_squared_error: 0.4123
Epoch 6/500
34/34 [==============================] - 0s - loss: 0.4332 - mean_squared_error: 0.4332
Epoch 7/500
34/34 [==============================] - 0s - loss: 0.4496 - mean_squared_error: 0.4496
Epoch 8/500
34/34 [==============================] - 0s - loss: 0.4388 - mean_squared_error

34/34 [==============================] - 0s - loss: 0.0050 - mean_squared_error: 0.0050
Epoch 67/500
34/34 [==============================] - 0s - loss: 0.0048 - mean_squared_error: 0.0048
Epoch 68/500
34/34 [==============================] - 0s - loss: 0.0046 - mean_squared_error: 0.0046
Epoch 69/500
34/34 [==============================] - 0s - loss: 0.0043 - mean_squared_error: 0.0043
Epoch 70/500
34/34 [==============================] - 0s - loss: 0.0041 - mean_squared_error: 0.0041
Epoch 71/500
34/34 [==============================] - 0s - loss: 0.0039 - mean_squared_error: 0.0039
Epoch 72/500
34/34 [==============================] - 0s - loss: 0.0038 - mean_squared_error: 0.0038
Epoch 73/500
34/34 [==============================] - 0s - loss: 0.0037 - mean_squared_error: 0.0037
Epoch 74/500
34/34 [==============================] - 0s - loss: 0.0036 - mean_squared_error: 0.0036
Epoch 75/500
34/34 [==============================] - 0s - loss: 0.0034 - mean_squared_error: 0.0034
Epo

34/34 [==============================] - 0s - loss: 3.4911e-04 - mean_squared_error: 3.4911e-04
Epoch 145/500
34/34 [==============================] - 0s - loss: 3.3820e-04 - mean_squared_error: 3.3820e-04
Epoch 146/500
34/34 [==============================] - 0s - loss: 3.2774e-04 - mean_squared_error: 3.2774e-04
Epoch 147/500
34/34 [==============================] - 0s - loss: 3.1749e-04 - mean_squared_error: 3.1749e-04
Epoch 148/500
34/34 [==============================] - 0s - loss: 3.0744e-04 - mean_squared_error: 3.0744e-04
Epoch 149/500
34/34 [==============================] - 0s - loss: 2.9764e-04 - mean_squared_error: 2.9764e-04
Epoch 150/500
34/34 [==============================] - 0s - loss: 2.8841e-04 - mean_squared_error: 2.8841e-04
Epoch 151/500
34/34 [==============================] - 0s - loss: 2.7970e-04 - mean_squared_error: 2.7970e-04
Epoch 152/500
34/34 [==============================] - 0s - loss: 2.7105e-04 - mean_squared_error: 2.7105e-04
Epoch 153/500
34/34 [===

34/34 [==============================] - 0s - loss: 2.3175e-05 - mean_squared_error: 2.3175e-05
Epoch 219/500
34/34 [==============================] - 0s - loss: 2.2312e-05 - mean_squared_error: 2.2312e-05
Epoch 220/500
34/34 [==============================] - 0s - loss: 2.1487e-05 - mean_squared_error: 2.1487e-05
Epoch 221/500
34/34 [==============================] - 0s - loss: 2.0689e-05 - mean_squared_error: 2.0689e-05
Epoch 222/500
34/34 [==============================] - 0s - loss: 1.9934e-05 - mean_squared_error: 1.9934e-05
Epoch 223/500
34/34 [==============================] - 0s - loss: 1.9182e-05 - mean_squared_error: 1.9182e-05
Epoch 224/500
34/34 [==============================] - 0s - loss: 1.8467e-05 - mean_squared_error: 1.8467e-05
Epoch 225/500
34/34 [==============================] - 0s - loss: 1.7776e-05 - mean_squared_error: 1.7776e-05
Epoch 226/500
34/34 [==============================] - 0s - loss: 1.7103e-05 - mean_squared_error: 1.7103e-05
Epoch 227/500
34/34 [===

34/34 [==============================] - 0s - loss: 1.2654e-06 - mean_squared_error: 1.2654e-06
Epoch 293/500
34/34 [==============================] - 0s - loss: 1.2167e-06 - mean_squared_error: 1.2167e-06
Epoch 294/500
34/34 [==============================] - 0s - loss: 1.1695e-06 - mean_squared_error: 1.1695e-06
Epoch 295/500
34/34 [==============================] - 0s - loss: 1.1234e-06 - mean_squared_error: 1.1234e-06
Epoch 296/500
34/34 [==============================] - 0s - loss: 1.0796e-06 - mean_squared_error: 1.0796e-06
Epoch 297/500
34/34 [==============================] - 0s - loss: 1.0381e-06 - mean_squared_error: 1.0381e-06
Epoch 298/500
34/34 [==============================] - 0s - loss: 9.9746e-07 - mean_squared_error: 9.9746e-07
Epoch 299/500
34/34 [==============================] - 0s - loss: 9.5858e-07 - mean_squared_error: 9.5858e-07
Epoch 300/500
34/34 [==============================] - 0s - loss: 9.2065e-07 - mean_squared_error: 9.2065e-07
Epoch 301/500
34/34 [===

...

34/34 [==============================] - 0s - loss: 6.2134e-05 - mean_squared_error: 6.2134e-05
Epoch 136/500
34/34 [==============================] - 0s - loss: 5.9506e-05 - mean_squared_error: 5.9506e-05
Epoch 137/500
34/34 [==============================] - 0s - loss: 5.7047e-05 - mean_squared_error: 5.7047e-05
Epoch 138/500
34/34 [==============================] - 0s - loss: 5.4782e-05 - mean_squared_error: 5.4782e-05
Epoch 139/500
34/34 [==============================] - 0s - loss: 5.2806e-05 - mean_squared_error: 5.2806e-05
Epoch 140/500
34/34 [==============================] - 0s - loss: 5.0847e-05 - mean_squared_error: 5.0847e-05
Epoch 141/500
34/34 [==============================] - 0s - loss: 4.8931e-05 - mean_squared_error: 4.8931e-05
Epoch 142/500
34/34 [==============================] - 0s - loss: 4.7031e-05 - mean_squared_error: 4.7031e-05
Epoch 143/500
34/34 [==============================] - 0s - loss: 4.5170e-05 - mean_squared_error: 4.5170e-05
Epoch 144/500
34/34 [===

34/34 [==============================] - 0s - loss: 2.4623e-06 - mean_squared_error: 2.4623e-06
Epoch 210/500
34/34 [==============================] - 0s - loss: 2.3442e-06 - mean_squared_error: 2.3442e-06
Epoch 211/500
34/34 [==============================] - 0s - loss: 2.2317e-06 - mean_squared_error: 2.2317e-06
Epoch 212/500
34/34 [==============================] - 0s - loss: 2.1233e-06 - mean_squared_error: 2.1233e-06
Epoch 213/500
34/34 [==============================] - 0s - loss: 2.0192e-06 - mean_squared_error: 2.0192e-06
Epoch 214/500
34/34 [==============================] - 0s - loss: 1.9191e-06 - mean_squared_error: 1.9191e-06
Epoch 215/500
34/34 [==============================] - 0s - loss: 1.8283e-06 - mean_squared_error: 1.8283e-06
Epoch 216/500
34/34 [==============================] - 0s - loss: 1.7394e-06 - mean_squared_error: 1.7394e-06
Epoch 217/500
34/34 [==============================] - 0s - loss: 1.6531e-06 - mean_squared_error: 1.6531e-06
Epoch 218/500
34/34 [===

34/34 [==============================] - 0s - loss: 3.7761e-08 - mean_squared_error: 3.7761e-08
Epoch 284/500
34/34 [==============================] - 0s - loss: 3.5503e-08 - mean_squared_error: 3.5503e-08
Epoch 285/500
34/34 [==============================] - 0s - loss: 3.3315e-08 - mean_squared_error: 3.3315e-08
Epoch 286/500
34/34 [==============================] - 0s - loss: 3.1179e-08 - mean_squared_error: 3.1179e-08
Epoch 287/500
34/34 [==============================] - 0s - loss: 2.9151e-08 - mean_squared_error: 2.9151e-08
Epoch 288/500
34/34 [==============================] - 0s - loss: 2.7235e-08 - mean_squared_error: 2.7235e-08
Epoch 289/500
34/34 [==============================] - 0s - loss: 2.5594e-08 - mean_squared_error: 2.5594e-08
Epoch 290/500
34/34 [==============================] - 0s - loss: 2.3982e-08 - mean_squared_error: 2.3982e-08
Epoch 291/500
34/34 [==============================] - 0s - loss: 2.2455e-08 - mean_squared_error: 2.2455e-08
Epoch 292/500
34/34 [===

34/34 [==============================] - 0s - loss: 1.6867e-10 - mean_squared_error: 1.6867e-10
Epoch 358/500
34/34 [==============================] - 0s - loss: 1.5652e-10 - mean_squared_error: 1.5652e-10
Epoch 359/500
34/34 [==============================] - 0s - loss: 1.4377e-10 - mean_squared_error: 1.4377e-10
Epoch 360/500
34/34 [==============================] - 0s - loss: 1.3215e-10 - mean_squared_error: 1.3215e-10
Epoch 361/500
34/34 [==============================] - 0s - loss: 1.2016e-10 - mean_squared_error: 1.2016e-10
Epoch 362/500
34/34 [==============================] - 0s - loss: 1.1245e-10 - mean_squared_error: 1.1245e-10
Epoch 363/500
34/34 [==============================] - 0s - loss: 1.0410e-10 - mean_squared_error: 1.0410e-10
Epoch 364/500
34/34 [==============================] - 0s - loss: 9.4040e-11 - mean_squared_error: 9.4040e-11
Epoch 365/500
34/34 [==============================] - 0s - loss: 8.6178e-11 - mean_squared_error: 8.6178e-11
Epoch 366/500
34/34 [===

34/34 [==============================] - 0s - loss: 1.2319e-12 - mean_squared_error: 1.2319e-12
Epoch 432/500
34/34 [==============================] - 0s - loss: 1.1350e-12 - mean_squared_error: 1.1350e-12
Epoch 433/500
34/34 [==============================] - 0s - loss: 1.0884e-12 - mean_squared_error: 1.0884e-12
Epoch 434/500
34/34 [==============================] - 0s - loss: 1.1498e-12 - mean_squared_error: 1.1498e-12
Epoch 435/500
34/34 [==============================] - 0s - loss: 1.1133e-12 - mean_squared_error: 1.1133e-12
Epoch 436/500
34/34 [==============================] - 0s - loss: 9.8352e-13 - mean_squared_error: 9.8352e-13
Epoch 437/500
34/34 [==============================] - 0s - loss: 9.7689e-13 - mean_squared_error: 9.7689e-13
Epoch 438/500
34/34 [==============================] - 0s - loss: 9.3576e-13 - mean_squared_error: 9.3576e-13
Epoch 439/500
34/34 [==============================] - 0s - loss: 8.9707e-13 - mean_squared_error: 8.9707e-13
Epoch 440/500
34/34 [===

### Backward Ensemble

In [ ]:
ensembles_rev = []
for j in range(ENS_COUNT//2):
    print('Training Ensemble {} - \n'.format(j))
    chain_reverse = []
    
    X_feed = X_train
    y_t = y_train
    for i in range(1,y_t.shape[1]+1):
        print('Training data - ',i)
        y_now = y_t[:,[-i,]].copy()
        print('Shapes - \n X = {} \n Y = {}'.format(X_feed.shape,y_now.shape))
        
        node = create_c1node(X_feed,y_now)
        node.fit(X_feed,y_now,epochs=500,batch_size=50)
        print('Training of node {} complete\n'.format(i))
        
        chain_reverse.append(node)
        X_feed = np.append(X_feed,y_now,axis=1)
        
    ensembles_rev.append(chain_reverse)

Training Ensemble 0 - 

Training data -  1
Shapes - 
 X = (34, 30) 
 Y = (34, 1)
Epoch 1/500
34/34 [==============================] - 1s - loss: 1.2534 - mean_squared_error: 1.2534
Epoch 2/500
34/34 [==============================] - 0s - loss: 0.9474 - mean_squared_error: 0.9474
Epoch 3/500
34/34 [==============================] - 0s - loss: 0.7671 - mean_squared_error: 0.7671
Epoch 4/500
34/34 [==============================] - 0s - loss: 0.6724 - mean_squared_error: 0.6724
Epoch 5/500
34/34 [==============================] - 0s - loss: 0.6249 - mean_squared_error: 0.6249
Epoch 6/500
34/34 [==============================] - 0s - loss: 0.5915 - mean_squared_error: 0.5915
Epoch 7/500
34/34 [==============================] - 0s - loss: 0.5490 - mean_squared_error: 0.5490
Epoch 8/500
34/34 [==============================] - 0s - loss: 0.4958 - mean_squared_error: 0.4958
Epoch 9/500
34/34 [==============================] - 0s - loss: 0.4388 - mean_squared_error: 0.4388
Epoch 10/500
34/34 

34/34 [==============================] - 0s - loss: 0.0058 - mean_squared_error: 0.0058
Epoch 82/500
34/34 [==============================] - 0s - loss: 0.0055 - mean_squared_error: 0.0055
Epoch 83/500
34/34 [==============================] - 0s - loss: 0.0052 - mean_squared_error: 0.0052
Epoch 84/500
34/34 [==============================] - 0s - loss: 0.0049 - mean_squared_error: 0.0049
Epoch 85/500
34/34 [==============================] - 0s - loss: 0.0047 - mean_squared_error: 0.0047
Epoch 86/500
34/34 [==============================] - 0s - loss: 0.0045 - mean_squared_error: 0.0045
Epoch 87/500
34/34 [==============================] - 0s - loss: 0.0042 - mean_squared_error: 0.0042
Epoch 88/500
34/34 [==============================] - 0s - loss: 0.0040 - mean_squared_error: 0.0040
Epoch 89/500
34/34 [==============================] - 0s - loss: 0.0038 - mean_squared_error: 0.0038
Epoch 90/500
34/34 [==============================] - 0s - loss: 0.0037 - mean_squared_error: 0.0037
Epo

34/34 [==============================] - 0s - loss: 4.8016e-05 - mean_squared_error: 4.8016e-05
Epoch 159/500
34/34 [==============================] - 0s - loss: 4.4317e-05 - mean_squared_error: 4.4317e-05
Epoch 160/500
34/34 [==============================] - 0s - loss: 4.0822e-05 - mean_squared_error: 4.0822e-05
Epoch 161/500
34/34 [==============================] - 0s - loss: 3.7803e-05 - mean_squared_error: 3.7803e-05
Epoch 162/500
34/34 [==============================] - 0s - loss: 3.5029e-05 - mean_squared_error: 3.5029e-05
Epoch 163/500
34/34 [==============================] - 0s - loss: 3.2087e-05 - mean_squared_error: 3.2087e-05
Epoch 164/500
34/34 [==============================] - 0s - loss: 2.9713e-05 - mean_squared_error: 2.9713e-05
Epoch 165/500
34/34 [==============================] - 0s - loss: 2.7609e-05 - mean_squared_error: 2.7609e-05
Epoch 166/500
34/34 [==============================] - 0s - loss: 2.5456e-05 - mean_squared_error: 2.5456e-05
Epoch 167/500
34/34 [===

34/34 [==============================] - 0s - loss: 1.0872e-08 - mean_squared_error: 1.0872e-08
Epoch 233/500
34/34 [==============================] - 0s - loss: 9.0093e-09 - mean_squared_error: 9.0093e-09
Epoch 234/500
34/34 [==============================] - 0s - loss: 7.5409e-09 - mean_squared_error: 7.5409e-09
Epoch 235/500
34/34 [==============================] - 0s - loss: 5.9222e-09 - mean_squared_error: 5.9222e-09
Epoch 236/500
34/34 [==============================] - 0s - loss: 4.6814e-09 - mean_squared_error: 4.6814e-09
Epoch 237/500
34/34 [==============================] - 0s - loss: 3.6479e-09 - mean_squared_error: 3.6479e-09
Epoch 238/500
34/34 [==============================] - 0s - loss: 2.9585e-09 - mean_squared_error: 2.9585e-09
Epoch 239/500
34/34 [==============================] - 0s - loss: 2.4496e-09 - mean_squared_error: 2.4496e-09
Epoch 240/500
34/34 [==============================] - 0s - loss: 1.8359e-09 - mean_squared_error: 1.8359e-09
Epoch 241/500
34/34 [===

34/34 [==============================] - 0s - loss: 1.5575e-12 - mean_squared_error: 1.5575e-12
Epoch 307/500
34/34 [==============================] - 0s - loss: 1.3803e-12 - mean_squared_error: 1.3803e-12
Epoch 308/500
34/34 [==============================] - 0s - loss: 1.2327e-12 - mean_squared_error: 1.2327e-12
Epoch 309/500
34/34 [==============================] - 0s - loss: 1.1379e-12 - mean_squared_error: 1.1379e-12
Epoch 310/500
34/34 [==============================] - 0s - loss: 1.0110e-12 - mean_squared_error: 1.0110e-12
Epoch 311/500
34/34 [==============================] - 0s - loss: 8.7491e-13 - mean_squared_error: 8.7491e-13
Epoch 312/500
34/34 [==============================] - 0s - loss: 6.8651e-13 - mean_squared_error: 6.8651e-13
Epoch 313/500
34/34 [==============================] - 0s - loss: 5.8070e-13 - mean_squared_error: 5.8070e-13
Epoch 314/500
34/34 [==============================] - 0s - loss: 5.1534e-13 - mean_squared_error: 5.1534e-13
Epoch 315/500
34/34 [===

...

34/34 [==============================] - 0s - loss: 5.2643e-11 - mean_squared_error: 5.2643e-11
Epoch 257/500
34/34 [==============================] - 0s - loss: 4.6041e-11 - mean_squared_error: 4.6041e-11
Epoch 258/500
34/34 [==============================] - 0s - loss: 3.9996e-11 - mean_squared_error: 3.9996e-11
Epoch 259/500
34/34 [==============================] - 0s - loss: 3.4990e-11 - mean_squared_error: 3.4990e-11
Epoch 260/500
34/34 [==============================] - 0s - loss: 3.1419e-11 - mean_squared_error: 3.1419e-11
Epoch 261/500
34/34 [==============================] - 0s - loss: 2.8510e-11 - mean_squared_error: 2.8510e-11
Epoch 262/500
34/34 [==============================] - 0s - loss: 2.5356e-11 - mean_squared_error: 2.5356e-11
Epoch 263/500
34/34 [==============================] - 0s - loss: 2.2549e-11 - mean_squared_error: 2.2549e-11
Epoch 264/500
34/34 [==============================] - 0s - loss: 1.9798e-11 - mean_squared_error: 1.9798e-11
Epoch 265/500
34/34 [===

34/34 [==============================] - 0s - loss: 3.7800e-14 - mean_squared_error: 3.7800e-14
Epoch 331/500
34/34 [==============================] - 0s - loss: 3.4038e-14 - mean_squared_error: 3.4038e-14
Epoch 332/500
34/34 [==============================] - 0s - loss: 2.9858e-14 - mean_squared_error: 2.9858e-14
Epoch 333/500
34/34 [==============================] - 0s - loss: 3.2575e-14 - mean_squared_error: 3.2575e-14
Epoch 334/500
34/34 [==============================] - 0s - loss: 3.3594e-14 - mean_squared_error: 3.3594e-14
Epoch 335/500
34/34 [==============================] - 0s - loss: 3.3072e-14 - mean_squared_error: 3.3072e-14
Epoch 336/500
34/34 [==============================] - 0s - loss: 3.1452e-14 - mean_squared_error: 3.1452e-14
Epoch 337/500
34/34 [==============================] - 0s - loss: 2.7403e-14 - mean_squared_error: 2.7403e-14
Epoch 338/500
34/34 [==============================] - 0s - loss: 2.3850e-14 - mean_squared_error: 2.3850e-14
Epoch 339/500
34/34 [===

34/34 [==============================] - 0s - loss: 2.2387e-14 - mean_squared_error: 2.2387e-14
Epoch 405/500
34/34 [==============================] - 0s - loss: 1.4472e-14 - mean_squared_error: 1.4472e-14
Epoch 406/500
34/34 [==============================] - 0s - loss: 1.3741e-14 - mean_squared_error: 1.3741e-14
Epoch 407/500
34/34 [==============================] - 0s - loss: 1.3244e-14 - mean_squared_error: 1.3244e-14
Epoch 408/500
34/34 [==============================] - 0s - loss: 1.1259e-14 - mean_squared_error: 1.1259e-14
Epoch 409/500
34/34 [==============================] - 0s - loss: 1.0658e-14 - mean_squared_error: 1.0658e-14
Epoch 410/500
34/34 [==============================] - 0s - loss: 9.8745e-15 - mean_squared_error: 9.8745e-15
Epoch 411/500
34/34 [==============================] - 0s - loss: 9.2214e-15 - mean_squared_error: 9.2214e-15
Epoch 412/500
34/34 [==============================] - 0s - loss: 9.0385e-15 - mean_squared_error: 9.0385e-15
Epoch 413/500
34/34 [===

34/34 [==============================] - 0s - loss: 1.0475e-14 - mean_squared_error: 1.0475e-14
Epoch 479/500
34/34 [==============================] - 0s - loss: 9.8745e-15 - mean_squared_error: 9.8745e-15
Epoch 480/500
34/34 [==============================] - 0s - loss: 8.5161e-15 - mean_squared_error: 8.5161e-15
Epoch 481/500
34/34 [==============================] - 0s - loss: 9.2998e-15 - mean_squared_error: 9.2998e-15
Epoch 482/500
34/34 [==============================] - 0s - loss: 9.9267e-15 - mean_squared_error: 9.9267e-15
Epoch 483/500
34/34 [==============================] - 0s - loss: 9.5087e-15 - mean_squared_error: 9.5087e-15
Epoch 484/500
34/34 [==============================] - 0s - loss: 9.4565e-15 - mean_squared_error: 9.4565e-15
Epoch 485/500
34/34 [==============================] - 0s - loss: 8.3071e-15 - mean_squared_error: 8.3071e-15
Epoch 486/500
34/34 [==============================] - 0s - loss: 1.0188e-14 - mean_squared_error: 1.0188e-14
Epoch 487/500
34/34 [===

34/34 [==============================] - 0s - loss: 0.0043 - mean_squared_error: 0.0043
Epoch 57/500
34/34 [==============================] - 0s - loss: 0.0038 - mean_squared_error: 0.0038
Epoch 58/500
34/34 [==============================] - 0s - loss: 0.0035 - mean_squared_error: 0.0035
Epoch 59/500
34/34 [==============================] - 0s - loss: 0.0034 - mean_squared_error: 0.0034
Epoch 60/500
34/34 [==============================] - 0s - loss: 0.0034 - mean_squared_error: 0.0034
Epoch 61/500
34/34 [==============================] - 0s - loss: 0.0035 - mean_squared_error: 0.0035
Epoch 62/500
34/34 [==============================] - 0s - loss: 0.0035 - mean_squared_error: 0.0035
Epoch 63/500
34/34 [==============================] - 0s - loss: 0.0034 - mean_squared_error: 0.0034
Epoch 64/500
34/34 [==============================] - 0s - loss: 0.0032 - mean_squared_error: 0.0032
Epoch 65/500
34/34 [==============================] - 0s - loss: 0.0030 - mean_squared_error: 0.0030
Epo

### Mean Aggregation

In [ ]:
##Getting the output from the ensembles
y_full_fw = []
y_full_re = []

for k in range(ENS_COUNT//2):
    print("\n\nENSEMBLE {}:\n\n".format(k+1))
    X_feed_fw = X_test.copy()
    X_feed_re = X_test.copy()
    j = 0
    #Model chain
    y_pred_fw = []
    y_pred_re = []
    for i in range(len(chain_forward)):
        node1 = ensembles_fwd[k][i]
        node2 = ensembles_rev[k][i]
        print("Getting op from chain node ",j)
        node_name = "Node" + str(i)
        print("Shapes:\n X = {}".format(X_feed_fw.shape))
        output_fw = node1.predict(X_feed_fw)
        output_rev = node2.predict(X_feed_re)
        y_pred_fw.append(output_fw)
        y_pred_re.append(output_rev)
        X_feed_fw = np.append(X_feed_fw,output_fw,axis=1)
        X_feed_re = np.append(X_feed_re,output_rev,axis=1)
        j+=1
    y_full_fw.append(y_pred_fw)
    y_full_re.append(y_pred_re)
        #Updating X_feed
            

In [ ]:
##Calculating the mean
cross_chain_group = []
for j in range(ENS_COUNT//2):
    prediction = []
    y_pred_fw = y_full_fw[j].copy()
    y_pred_re = y_full_re[j].copy()
    fw = np.array(y_pred_fw)[:,:,0].transpose().copy()
    re = np.array(y_pred_re)[:,:,0].transpose().copy()
    for i in range(np.array(y_pred_fw)[:,:,0].transpose().shape[0]):
        fw_i = fw[i,:].reshape(1,-1)[0]
        re_i = re[i,:].reshape(1,-1)[0][::-1] 
        final = (fw_i+re_i)/2
        prediction.append(final)
    cross_chain_group.append(np.array(prediction))

In [ ]:
#Adding all results
total = None
for x in cross_chain_group:
    if total is None:
        total = x
    else:
        total = total + x
#Calculating the average
predictions = (total/(ENS_COUNT//2))

### Evaluating Mean Aggregation

In [ ]:
evaluate(scaler.inverse_transform(y_test),scaler.inverse_transform(np.array(predictions)))